# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [3]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to yangdong07@gmail.com and will expire on August 28, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1505125022.log


In [5]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [6]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 2.221317     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 2.292828     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
print ('number of nonzeros = %d' % (model_all.coefficients['value']).nnz())

coef = model_all.coefficients

coef[coef['value'] != 0][:]

number of nonzeros = 6


name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np
def RSS(y_true, y_predict):
    residual = y_true - y_predict
    return np.dot(residual, residual)


def print_coefficient(model):
#     print(model)
    coef = model.coefficients
    print ('number of nonzeros = %d' % (coef['value']).nnz())
    print(coef[coef['value'] != 0][:])
    
    
def train_model(data, target, l1_penalty, max_iterations=1000, verbose=False):
    model = graphlab.linear_regression.create(
        data, target=target, features=all_features,
        validation_set=None, verbose=verbose,
        l2_penalty=0., l1_penalty=l1_penalty,
        max_iterations=max_iterations)
    return model

In [21]:
model = train_model(training, 'price', 1e5, max_iterations=3000, verbose=True)
print_coefficient(model)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 3000      | 3001     | 0.000003  | 151.645185   | 2802777.159518     | 207709.825145 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

number of nonzeros = 18
+------------------+-------+----------------+--------+
|       name       | index |     value      | stderr |
+------------------+-------+----------------+--------+
|   (intercept)    |  None | 7516152.69034  |  None  |
|     bedrooms     |  None | -17156.423975  |  None  |
| bedrooms_square  |  None | 530.831989588  |  None  |
|    bathrooms     |  None | 60813.2452288  |  None  |
|   sqft_living    |  None |  316.83337139  |  None  |
| sqft_living_sqrt |  None | -51290.4847285 |  None  |
|     sqft_lot     |  None | 1.03582659941  |  None  |
|  sqft_lot_sqrt   |  None | -858.684962183 |  None  |
|      floors      |  None | -86768.2629607 |  None  |
|  floors_square   |  None | 30286.9416622  |  None  |
+------------------+-------+----------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [24]:
np.logspace(1, 7, num=13)

array([  1.00000000e+01,   3.16227766e+01,   1.00000000e+02,
         3.16227766e+02,   1.00000000e+03,   3.16227766e+03,
         1.00000000e+04,   3.16227766e+04,   1.00000000e+05,
         3.16227766e+05,   1.00000000e+06,   3.16227766e+06,
         1.00000000e+07])

In [59]:

import time

models = dict()
rss = dict()
for l1_penalty in np.logspace(1, 7, num=13):
    start = time.time()
    model = train_model(training, 'price', l1_penalty, max_iterations=5000)
    models[l1_penalty] = model
    rss[l1_penalty] = RSS(validation['price'], model.predict(validation))
    print('%12.1f\t%3d\t%10.4g\t %.0f s' % (l1_penalty, model.coefficients['value'].nnz(),
                                           rss[l1_penalty], (time.time() - start)))
    
#     print_coefficient(model)
    

        10.0	 18	 4.033e+14	 221 ms
        31.6	 18	 4.033e+14	 213 ms
       100.0	 18	 4.033e+14	 214 ms
       316.2	 18	 4.033e+14	 225 ms
      1000.0	 18	 4.033e+14	 216 ms
      3162.3	 18	 4.033e+14	 211 ms
     10000.0	 18	 4.033e+14	 211 ms
     31622.8	 18	 4.033e+14	 198 ms
    100000.0	 18	 4.032e+14	 202 ms
    316227.8	 18	 4.029e+14	 196 ms
   1000000.0	 18	 4.022e+14	 205 ms
   3162277.7	 18	 4.014e+14	 202 ms
  10000000.0	 15	 4.144e+14	 193 ms


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [1]:
min(rss.items(), key=lambda x: x[1])

NameError: name 'rss' is not defined

In [44]:
# print(rss[100000])
# print(RSS(validation['price'], model.predict(validation)))


for model in models.values():
    print_coefficient(model)

number of nonzeros = 9
+---------------+-------+----------------+--------+
|      name     | index |     value      | stderr |
+---------------+-------+----------------+--------+
|  (intercept)  |  None | -264517.238331 |  None  |
|  sqft_living  |  None | 210.665336347  |  None  |
| sqft_lot_sqrt |  None | -131.521118505 |  None  |
|   waterfront  |  None | 570295.535475  |  None  |
|      view     |  None | 68959.2743556  |  None  |
|   condition   |  None | 3842.98013917  |  None  |
|     grade     |  None |  44489.484322  |  None  |
| sqft_basement |  None | 2.97326517413  |  None  |
|  yr_renovated |  None | 49.5925234494  |  None  |
+---------------+-------+----------------+--------+
[9 rows x 4 columns]

number of nonzeros = 6
+--------------+-------+---------------+--------+
|     name     | index |     value     | stderr |
+--------------+-------+---------------+--------+
| (intercept)  |  None | 21945.1252773 |  None  |
| sqft_living  |  None |  234.41225288 |  None  |
|  wat

+---------------+-------+----------------+--------+
|      name     | index |     value      | stderr |
+---------------+-------+----------------+--------+
|  (intercept)  |  None | -96735.0193961 |  None  |
|  sqft_living  |  None |  227.46290368  |  None  |
| sqft_lot_sqrt |  None | -46.6999488088 |  None  |
|   waterfront  |  None | 548697.967539  |  None  |
|      view     |  None | 71004.1111866  |  None  |
|     grade     |  None | 18757.6314667  |  None  |
|  yr_renovated |  None |  42.227672297  |  None  |
+---------------+-------+----------------+--------+
[7 rows x 4 columns]

number of nonzeros = 4
+-------------+-------+---------------+--------+
|     name    | index |     value     | stderr |
+-------------+-------+---------------+--------+
| (intercept) |  None | 160176.779359 |  None  |
| sqft_living |  None | 173.518797494 |  None  |
|  waterfront |  None | 332445.771511 |  None  |
|     view    |  None | 71271.1257824 |  None  |
+-------------+-------+---------------+-

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [11]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [13]:
model = train_model(training, 'price', 1e8, max_iterations=1000, verbose=True)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 975       | 976      | 0.000003  | 43.390352    | 4541565.431497     | 243073.260954 |

| 1000      | 1001     | 0.000003  | 44.571664    | 4541062.631137     | 243059.819851 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [15]:
print_coefficient(model)

models = {}

models[1e8] = model

number of nonzeros = 9
+---------------+-------+----------------+--------+
|      name     | index |     value      | stderr |
+---------------+-------+----------------+--------+
|  (intercept)  |  None | -269929.877343 |  None  |
|  sqft_living  |  None | 209.843324063  |  None  |
| sqft_lot_sqrt |  None | -130.492150814 |  None  |
|   waterfront  |  None | 570332.060701  |  None  |
|      view     |  None | 68849.3074269  |  None  |
|   condition   |  None | 4109.62267543  |  None  |
|     grade     |  None | 45272.2684584  |  None  |
| sqft_basement |  None | 3.36352578278  |  None  |
|  yr_renovated |  None | 49.6744855882  |  None  |
+---------------+-------+----------------+--------+
[9 rows x 4 columns]



In [18]:
for l1_penalty in l1_penalty_values:
    model = train_model(training, 'price', l1_penalty, max_iterations=1000)
    print(l1_penalty)
    models[l1_penalty] = model

100000000.0
127427498.57
162377673.919
206913808.111
263665089.873
335981828.628
428133239.872
545559478.117
695192796.178
885866790.41
1128837891.68
1438449888.29
1832980710.83
2335721469.09
2976351441.63
3792690190.73
4832930238.57
6158482110.66
7847599703.51
10000000000.0


In [21]:

from collections import OrderedDict

order_models = OrderedDict()

for l1_penalty in l1_penalty_values:
    order_models[l1_penalty] = models[l1_penalty]
    
for l1_penalty, model in order_models.iteritems():
    coef = model.coefficients
    print(l1_penalty, coef['value'].nnz())

100000000.0 9
127427498.57 7
162377673.919 7
206913808.111 6
263665089.873 5
335981828.628 5
428133239.872 5
545559478.117 4
695192796.178 4
885866790.41 4
1128837891.68 4
1438449888.29 4
1832980710.83 3
2335721469.09 2
2976351441.63 1
3792690190.73 1
4832930238.57 1
6158482110.66 1
7847599703.51 1
10000000000.0 1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [24]:
l1_penalty_min = l1_penalty_values[0]
l1_penalty_max = l1_penalty_values[3]

print(l1_penalty_min)
print(l1_penalty_max)

100000000.0
206913808.111


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [25]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [30]:
for l1_penalty in l1_penalty_values:
    model = train_model(training, 'price', l1_penalty, max_iterations=2000)
    print(l1_penalty, model.coefficients['value'].nnz())
    models[l1_penalty] = model

100000000.0 9
105627042.532 9
111254085.064 8
116881127.597 7
122508170.129 7
128135212.661 7
133762255.193 7
139389297.725 7
145016340.257 7
150643382.79 7
156270425.322 7
161897467.854 7
167524510.386 8
173151552.918 8
178778595.451 8
184405637.983 7
190032680.515 6
195659723.047 6
201286765.579 6
206913808.111 6


In [37]:

order_models2 = OrderedDict()

for l1_penalty in l1_penalty_values:
    if models[l1_penalty].coefficients['value'].nnz() == 7:
        order_models2[l1_penalty] = models[l1_penalty]
    
validation_error = dict()
for l1_penalty, model in order_models2.iteritems():
    coef = model.coefficients
    validation_error[l1_penalty] = RSS(validation['price'], model.predict(validation))
    print(l1_penalty, coef['value'].nnz())

116881127.597 7
122508170.129 7
128135212.661 7
133762255.193 7
139389297.725 7
145016340.257 7
150643382.79 7
156270425.322 7
161897467.854 7
184405637.983 7


In [43]:
best = min(validation_error.items(), key=lambda x: x[1])

best_model = order_models2[best[0]]
print(best)
print_coefficient(best_model)

(116881127.59654932, 514361075626750.81)
number of nonzeros = 7
+---------------+-------+----------------+--------+
|      name     | index |     value      | stderr |
+---------------+-------+----------------+--------+
|  (intercept)  |  None | -194613.462678 |  None  |
|  sqft_living  |  None | 217.389965566  |  None  |
| sqft_lot_sqrt |  None | -104.361115248 |  None  |
|   waterfront  |  None | 562871.451701  |  None  |
|      view     |  None | 70083.6846348  |  None  |
|     grade     |  None | 35004.1439378  |  None  |
|  yr_renovated |  None | 46.8766143558  |  None  |
+---------------+-------+----------------+--------+
[7 rows x 4 columns]



***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?